# Assignment 1: Build a parser and tokenizer for Wikipedia content







## General guidelines

This notebook contains considerable amount of code to help you complete this assignment. Your task is to implement any missing parts of the code and answer any questions (if exist) within this notebook. This will require understanding the existing code, may require reading about packages being used, reading additional resources, and maybe even going over your notes from class 😱

**Evaluation and auto-grading**: Your submissions will be evaluated using both automatic and manual grading. Code parts for implementation are marked with a comment `# YOUR CODE HERE`, and usually followed by cell(s) containing automatic tests that evaluate the correctness of your answer. Staff will allow your notebook to **execute from start to finish for no more than 90 seconds**, then manually assess your submission. Any automatic tests that did not run due to your notebook timing out **will automatically receive 0 points**. The execution time excludes initial data download, which will already exist in the testing environment. The staff reserves the right to **modify any grade provided by the auto-grader** as well as to **execute additional tests not provided to you**. It is also important to note that **auto-graded cells only result in full or no credit**. In other words, you must pass all tests implemented in a test cell in order to get the credit for it, and passing some, but not all, of the tests in a test cell will not earn you any points for that cell. 

**Submission**: Unless specified otherwise, you need to upload this notebook file **with your ID as the file name**, e.g. 012345678.ipynb, to the assignment on Moodle. Before submitting, **make sure the notebook executes from start to finish in less than 90 seconds**.


# Tasks

In this assignment, we are going to read, parse, and tokenize a small number of Wikipedia articles from a single part file, and learn how to collect and merge page view information with articles. By the end of this assignment, you will be able to:

1. (25 Points) Parse and clean Wikipedia page entries from one part file of the wiki dump. From each page, extract the title, body, and a list of (anchor text, page title) pairs for links to other English Wikipedia pages. 
2. (70 Points) Tokenize the text of Wikipedia articles using a regex tokenizer. 
3. (5 Points) Collect page view information from Wikipedia and merge that with articles' data.




In [ ]:
import numpy as np
import bz2
from functools import partial
from collections import Counter
import pickle
from itertools import islice
from xml.etree import ElementTree
import codecs
import csv
import time
import os
import re
import gdown
from pathlib import Path

# 1. Parsing a dump file


First, let's download the file and examine it a bit.

In [ ]:
## Download one wikipedia file
path_url = 'https://drive.google.com/file/d/1c2ggRHG0WqLmJpIE-HfQgJi73kww3Zyc/view?usp=sharing'
wiki_file = 'enwiki-20210801-pages-articles-multistream15.xml-p17324603p17460152.bz2'
id = '1c2ggRHG0WqLmJpIE-HfQgJi73kww3Zyc'
gdown.download(id=id, output=wiki_file, quiet=False)

In [ ]:
# Make sure you downloaded the file
!ls

In [ ]:
# Uncomment this to view the first 59 lines of the (uncompressed) file
#!bzcat $wiki_file | head -n 59

## Parsing from XML

The following code reads the wiki dump file, parses its XML, and iterate over pages. First, it filters out pages that are redirects to other pages, talk pages, and any other pages that are not articles pages. Then, it extracts the article id and article markup text.

**YOUR TASK (5 Points)**: Implement the extraction of article title from the XML. 

In [ ]:
def page_iter(wiki_file):
  """ Reads a wiki dump file and create a generator that yields pages. 
  Parameters:
  -----------
  wiki_file: str
    A path to wiki dump file.
  Returns:
  --------
  tuple
    containing three elements: article id, title, and body. 
  """
  # open compressed bz2 dump file
  with bz2.open(wiki_file, 'rt', encoding='utf-8', errors='ignore') as f_in:
    # Create iterator for xml that yields output when tag closes
    elems = (elem for _, elem in ElementTree.iterparse(f_in, events=("end",)))
    # Consume the first element and extract the xml namespace from it. 
    # Although the raw xml has the  short tag names without namespace, i.e. it 
    # has <page> tags and not <http://wwww.mediawiki.org/xml/export...:page> 
    # tags, the parser reads it *with* the namespace. Therefore, it needs the 
    # namespace when looking for child elements in the find function as below.
    elem = next(elems)
    m = re.match("^{(http://www\.mediawiki\.org/xml/export-.*?)}", elem.tag)
    if m is None:
        raise ValueError("Malformed MediaWiki dump")
    ns = {"ns": m.group(1)}
    page_tag = ElementTree.QName(ns['ns'], 'page').text
    # iterate over elements
    for elem in elems:
      if elem.tag == page_tag:
        # Filter out redirect and non-article pages
        if elem.find('./ns:redirect', ns) is not None or \
           elem.find('./ns:ns', ns).text != '0':
          elem.clear()
          continue
        # Extract the article wiki id
        wiki_id = elem.find('./ns:id', ns).text
        # Extract the article title into a variables called title
        # YOUR CODE HERE
        raise NotImplementedError()
        # extract body
        body = elem.find('./ns:revision/ns:text', ns).text

        yield wiki_id, title, body
        elem.clear()

In [ ]:
# Print the first page
p1 = next(page_iter(wiki_file))
print(f"{p1[0]} {p1[1]}\n\n{' '*80}\n{p1[2]}")

In [ ]:
# Check the title of the first article
assert p1[1] == 'Langnes'

## Parsing articles (MediaWiki)

Wikipedia articles are written in a special markdown format called [MediaWiki](https://en.wikipedia.org/wiki/MediaWiki). The format is described [here](https://www.mediawiki.org/wiki/Help:Formatting) and you will need to read a little bit about it in order to complete this assignment successfully. A key property of the MediaWiki markdown is that it's recursive -- markdown can be (and often does) nest inside other markdown elements. For example, a wiki link can contain another wiki link:
```
[[File:image1.jpg|[[Wikipedia]]]]
```

Fortunately, there are implementations of parsers for MediaWiki, and we are going to use one called `mwparserfromhell`. Let's import/install it first. 

In [ ]:
try:
    import mwparserfromhell as mwp
except ImportError:
    !pip install -I mwparserfromhell==0.6.0
finally:
    import mwparserfromhell as mwp
# modify the parser behavior a bit, no need to understand this code.
mwp.definitions.INVISIBLE_TAGS.append('ref')

Let's parse some MediaWiki and look at the result:

In [ ]:
wikicode = mwp.parse('Lorem ipsum {{foo|bar|{{baz}}|spam=eggs}}')
print(wikicode.get_tree())

Naturally, the result of parsing the MediaWiki format results is **a tree**. In the above case, the markdown contains a [template](https://www.mediawiki.org/wiki/Help:Templates) named foo, which takes two positional parameters, another template called baz, and a named parameter called spam. Templates are both cool (in providing structured information) and wild (in the sense that their structure keeps changing). 

A particular type of template called **Infobox** is the backbone of some of the largest knowledge bases on the planet(!), powering DBPedia and many popular IR applications such as Apple's Siri, Google Search, and more. To get a sense of the richness of infoboxes, let's look at one.

In [ ]:
iter = page_iter(wiki_file)
next(iter)
p2 = next(iter)
wikicode = mwp.parse(p2[2], skip_style_tags=True)
for template in wikicode.ifilter_templates():
  if str(template.name).strip().startswith('Infobox'):
    print(template)

Look at all this beautifully-structured data about a random village in Norway!!!

## Parsing outgoing article links (wiki links)

Putting aside templates and infoboxes, one of the key elements we care about in search engines is links. The text of the link, also known as anchor text, is a very useful description of the page that is being linked. Analyzing the link structure between pages using algorithms like PageRank, which we will cover later in class, is one of the key factors for improving search results. 

In this assignment, we're going to focus on links from one wikipedia article to another wikipedia article.

**YOUR TASK (20 POINTS)**: Complete the impementation of `get_wikilinks` and `filter_article_links` to return a list of (link, anchor text) for all outgoing links from an article to other wikipedia articles that it points to. See MediaWiki's format for [internal links](https://www.mediawiki.org/wiki/Help:Links#Internal_links). Please remove from the link any reference to a section/anchor in the target page. You are free to implement your filter through a regex or some other means. 

In [ ]:
def filter_article_links(title):
  """ Return false for wikilink titles (str) pointing to non-articles such as images, files, media and more (as described in the documentation).
      Otherwise, returns true. 
  """
  # YOUR CODE HERE
  raise NotImplementedError()

def get_wikilinks(wikicode):
  """ Traverses the parse tree for internal links and filter out non-article 
  links.
  Parameters:
  -----------
  wikicode: mwp.wikicode.Wikicode
    Parse tree of some WikiMedia markdown.
  Returns:
  --------
  list of (link: str, anchor_text: str) pair
    A list of outgoing links from the markdown to wikipedia articles.
  """
  links = []
  for wl in wikicode.ifilter_wikilinks():
    # skip links that don't pass our filter
    title = str(wl.title)
    if not filter_article_links(title):
      continue
    # if text is None use title, otherwise strip markdown from the anchor text.
    text = wl.text
    if text is None:
      text = title
    else:
      text = text.strip_code()
    # remove any lingering section/anchor reference in the link
    # YOUR CODE HERE
    raise NotImplementedError()
    links.append((title, text))
  return links

In [ ]:
# Basic checks that we can extract links correctly
get_wl = lambda text: get_wikilinks(mwp.parse(text))
assert get_wl("[[Wikipedia]]")[0] == ('Wikipedia', 'Wikipedia')
assert get_wl("[[Wikipedia|some text]]")[0] == ('Wikipedia', 'some text')
assert len(get_wl("[[File:example.jpg|frame|caption]]")) == 0
assert get_wl('[[Wikipedia#Preview|preview]]')[0] == ('Wikipedia', 'preview')

In [ ]:
# More advanced tests
assert len(get_wl(p2[2])) < len(mwp.parse(p2[2]).filter_wikilinks())
pages = list(islice(page_iter(wiki_file), None, 25))
p4, p14, p24 = pages[4], pages[14], pages[24]
assert len(get_wl(p4[2])) == 32
assert len(get_wl(p14[2])) == 891
assert len(get_wl(p24[2])) == 25

# 2. Tokenization

Before tokenizing Wikipedia articles' text we need to remove any remaining MediaWiki markdown from the text. Luckily, our parser knows how to strip all markdown as demonstrated by the following example:

In [ ]:
def remove_markdown(text):
  return mwp.parse(text).strip_code()
print(remove_markdown("""
== Section 2 ==
[[File:image1.jpg| '''''beautiful''''' <b>image</b> of [[Wikipedia]]]]
"""))

Great! now we can focus on tokenzing the clean text. Here's the clean text of one article after preprocessing:

In [ ]:
print(remove_markdown(p4[2]))

**YOUR TASK (70 POINTS)**: Complete the implementation of the functions in the next cell that return regular expressions (as strings) to capture dates, time, etc. in the text. 

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
RE_TOKENIZE = re.compile(rf"""
(
    # parsing html tags
     (?P<HTMLTAG>{get_html_pattern()})                                  
    # dates
    |(?P<DATE>{get_date_pattern()})
    # time
    |(?P<TIME>{get_time_pattern()})
    # Percents
    |(?P<PERCENT>{get_percent_pattern()})
    # Numbers
    |(?P<NUMBER>{get_number_pattern()})
    # Words
    |(?P<WORD>{get_word_pattern()})
    # space
    |(?P<SPACE>[\s\t\n]+) 
    # everything else
    |(?P<OTHER>.))""",  re.MULTILINE | re.IGNORECASE | re.VERBOSE | re.UNICODE)

def tokenize(text):
  return [(v, k) for match in RE_TOKENIZE.finditer(text)
                 for k, v in match.groupdict().items() 
                 if v is not None and k != 'SPACE']

In [ ]:
# html basic tests (7 points)
def tok(text):
  return tokenize(remove_markdown(text))
tokens = tok(r'<nowiki><b>hello</b></nowiki>')
assert ('<b>', 'HTMLTAG') in tokens
assert ('</b>', 'HTMLTAG') in tokens
tokens = tok(r'<nowiki><b style="color:red">hello</b></nowiki>')
assert ('<b style="color:red">', 'HTMLTAG') in tokens
tokens = tok(r'<nowiki><br /></nowiki>')
assert ('<br />', 'HTMLTAG') in tokens

In [ ]:
# html advanced tests (6 points)
tokens = tok(r'<nowiki><b><i>hello</i></b></nowiki>')
assert 4 == sum([1 for _, t in tokens if t == 'HTMLTAG'])

In [ ]:
# dates basic tests (7 points)
tokens = tok(r'dates in the format of January 29, 1984, Nov 3, 2020, or 3 Nov 2020.')
assert ('January 29, 1984', 'DATE') in tokens
assert ('Nov 3, 2020', 'DATE') in tokens
assert ('3 Nov 2020', 'DATE') in tokens

In [ ]:
# dates advanced tests (6 points)
tokens = tok(r'Sep 29, 1984, Apr 33, 2020, or 30 feb 2020.')
assert ('Sep 29, 1984', 'DATE') in tokens
assert ('Apr 33, 2020', 'DATE') not in tokens
assert ('30 feb 2020', 'DATE') not in tokens

In [ ]:
# time basic tests (7 points)
tokens = tok(r'12.12PM 1202a.m. 6:12:12')
assert ('12.12PM', 'TIME') in tokens
assert ('1202a.m.', 'TIME') in tokens
assert ('6:12:12', 'TIME') in tokens

In [ ]:
# time advanced tests (6 points)
tokens = tok(r'36.12PM 1272a.m. 1202a.m 12:12:12am 56:12:12 6:72:12')
assert 0 == sum([1 for _, t in tokens if t == 'TIME'])

In [ ]:
# number basic tests (7 points)
tokens = tok(r"""12 +12 -12.0 -12,345.5466 +12,345,678,678 0.154""")
assert ('12', 'NUMBER') in tokens
assert ('+12', 'NUMBER') in tokens
assert ('-12.0', 'NUMBER') in tokens
assert ('-12,345.5466', 'NUMBER') in tokens
assert ('+12,345,678,678', 'NUMBER') in tokens
assert ('0.154', 'NUMBER') in tokens

In [ ]:
# number advanced tests (6 points)
assert ('500', 'NUMBER') in tok('the pound (500 in value)...')
assert ('500', 'NUMBER') in tok('the price is 500.')
assert ('500', 'NUMBER') in tok('the price is 500, but it is negotiable.')
assert ('500', 'NUMBER') in tok('the price is 500: no less!')
assert ('500', 'NUMBER') not in tok('the price rose 500%')
tokens = tok(r"""12.A W12 +-12 -.12.0 -12,34.5466 +12,345,6+78,678 0.15,4""")
assert 0 == sum([1 for _, t in tokens if t == 'NUMBER'])

In [ ]:
# word tests (13 points)
tokens = tok(r"""Hello Bob! It's Mary, your mother-in-law, 
  the mistake is your parents'! --Mom""")
assert ('Hello', 'WORD') in tokens
assert ('Bob', 'WORD') in tokens
assert ("It's", 'WORD') in tokens
assert ('Mary', 'WORD') in tokens
assert ('your', 'WORD') in tokens
assert ('mother-in-law', 'WORD') in tokens
assert ("parents'", 'WORD') in tokens
assert ("Mom", 'WORD') not in tokens
assert ("-Mom", 'WORD') not in tokens
assert ("--Mom", 'WORD') not in tokens

In [ ]:
# comprehensiveness test (5 points)
_, t = zip(*tok(pages[9][2]))
assert 5 == len(set(t))

# 3. Collect and merge page views

Data about page views on Wikipedia is available at https://dumps.wikimedia.org and there is documentation about the [definition of a page view](https://meta.wikimedia.org/wiki/Research:Page_view) and the [format of lines](https://dumps.wikimedia.org/other/pagecounts-ez/) in the file. In the class project, you will need to use page view data that we'll provide for ALL of English Wikipedia from the month of August 2021, which is more than 10.7 million viewed articles. The commented out code below shows how we generate that data, no need to run it yourself, this is just for your information. 

In [ ]:
# # Paths
# # Using user page views (as opposed to spiders and automated traffic) for the 
# # month of August 2021
# pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
# p = Path(pv_path) 
# pv_name = p.name
# pv_temp = f'{p.stem}-4dedup.txt'
# pv_clean = f'{p.stem}.pkl'
# # Download the file (2.3GB) 
# !wget -N $pv_path
# # Filter for English pages, and keep just two fields: article ID (3) and monthly 
# # total number of page views (5). Then, remove lines with article id or page 
# # view values that are not a sequence of digits.
# !bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp
# # Create a Counter (dictionary) that sums up the pages views for the same 
# # article, resulting in a mapping from article id to total page views.
# wid2pv = Counter()
# with open(pv_temp, 'rt') as f:
#   for line in f:
#     parts = line.split(' ')
#     wid2pv.update({int(parts[0]): int(parts[1])})
# # write out the counter as binary file (pickle it)
# with open(pv_clean, 'wb') as f:
#   pickle.dump(wid2pv, f)
# # read in the counter
# # with open(pv_clean, 'rb') as f:
# #   wid2pv = pickle.loads(f.read())

In order to keep things simple, in this assignment we provide you with a small sample of articles and their page view counts.

**YOUR TASK (5 POINTS)**: Complete the implementation of `most_viewed` for ranking articles from the most viewed to the least viewed.

In [ ]:
# A counter mapping article id to number of page views
wid2pv = Counter({
    '17324616': 10, '17324662': 4, '17324672': 16, '17324677': 612, 
    '17324689': 66, '17324702': 274, '17324704': 49, '17324943': 76, 
    '17324721': 35, '17324736': 2801, '17324747': 641, '17324758': 33,
    '17324768': 26, '17324783': 28, '17324788': 575, '17324790': 43, 
    '17324802': 29, '17324816': 159, '17324818': 57, '17324823': 60,
    '17324834': 19, '17324835': 7, '17324893': 116, '17324908': 2038,
    '15580374': 181126232, '1610886': 4657885, '30635': 8143874, 
    '3390': 4525604, '49632909': 5027640, '51150040': 3284643, 
    '60827': 4323859, '623737': 3427102, '65984422': 3733064, '737': 6039676
})

def most_viewed(pages):
  """Rank pages from most viewed to least viewed using the above `wid2pv` 
     counter.
  Parameters:
  -----------
    pages: An iterable list of pages as returned from `page_iter` where each 
           item is an article with (id, title, body)
  Returns:
  --------
  A list of tuples
    Sorted list of articles from most viewed to least viewed article with 
    article title and page views. For example:
    [('Langnes, Troms': 16), ('Langenes': 10), ('Langenes, Finnmark': 4), ...]
  """
  # YOUR CODE HERE
  raise NotImplementedError()

In [ ]:
pages_ranked = most_viewed(pages)
_, views = zip(*pages_ranked)
assert 25 == len(pages_ranked)
assert ('No W', 274) in pages_ranked
assert 7774 == sum(views)
assert 2226 == views[0] - views[4]
assert 10 == round(sum(views[0:5]) / sum(views[5:10]))